# WeatherPy - Hotel Search

### Dependencies and data

In [1]:
%matplotlib inline

import datetime as dt
import gmaps
import requests
import numpy as np
import pandas as pd

import config

In [2]:
# Configure API key
gmaps.configure(api_key=config.GMAPS_API_KEY)

In [3]:
# Weather data
weather = pd.read_csv('data/weather.csv')
weather.head(2)

,ID,City,Country,Latitude,Longitude,Time,Description,Max Temp,Humidity,Wind Speed,Cloudiness,Rain,Snow
0,0,Garoowe,SO,8.4054,48.4845,2021-01-09 20:45:48,clear sky,66.99,64,16.51,0,0.0,0.0
1,1,Vendas Novas,PT,38.6771,-8.4579,2021-01-09 20:45:49,few clouds,43.00,65,11.50,20,0.0,0.0


### Find hotels near destinations within the user's preferred weather

In [4]:
# Get the lowest and highest `Max Temp` in the data
min_mt, max_mt = weather['Max Temp'].min(), weather['Max Temp'].max()
min_mt, max_mt

(-47.2, 96.01)

In [5]:
def take_input(prompt, min_temp=min_mt, max_temp=max_mt, yes_no=False):
    
    """
    Take user input for weather preferences.
    
    Args:
        [1] prompt (str) - input prompt
        [2] min_temp (float) - minimum temperature in data
        [3] max_temp (float) - maximum temperature in data
        [4] yes_no (bool) - whether the input should be "yes" or "no" only
        
    Returns:
        [float or str] User input
    """
    
    # Initialize user input and loop condition to False
    i = b = False
    
    if yes_no: # yes/no input
        while i not in ['yes', 'no']: # continue until input is "yes" or "no"
            i = input(prompt) # take user input
        i = 1 if i == 'yes' else 0 # convert input to numeric
    else: 
        while not b: # loop condition
            try:
                i = float(input(prompt)) # take user input and convert to numeric
            except:
                continue
            if min_temp <= i <= max_temp: # check if user input is within proper temperature range
                b = True # end loop
            
    # Numeric input
    return i
        
    
# Get preferred weather
min_temp = take_input('What is the lowest temperature you would like at your destination? ')
max_temp = take_input('What is the highest temperature you would like at your destination? ', min_temp=min_temp)
rain = take_input('Would you like it to rain at your destination? (yes/no) ', yes_no=True)
snow = take_input('Would you like it to snow at your destination? (yes/no) ', yes_no=True)
min_temp, max_temp, rain, snow

What is the lowest temperature you would like at your destination? 70
What is the highest temperature you would like at your destination? 90
Would you like it to rain at your destination? (yes/no) yes
Would you like it to snow at your destination? (yes/no) no


(70.0, 90.0, 1, 0)

In [6]:
# Filter for max temp preference
pref_cities = weather.loc[(weather['Max Temp'] >= min_temp) & (weather['Max Temp'] <= max_temp)]

# Filter for rain preference
if rain:
    pref_cities = pref_cities[pref_cities['Rain'] > 0]
else:
    pref_cities = pref_cities[pref_cities['Rain'] == 0]

# Filter for snow preference
if snow:
    pref_cities = pref_cities[pref_cities['Snow'] > 0]
else:
    pref_cities = pref_cities[pref_cities['Snow'] == 0]
    
pref_cities.shape

(70, 13)

In [7]:
# Create new dataframe
hotels = pref_cities.copy()
hotels['Hotel'] = np.NaN # create new col for hotels

# Request url and params
base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'
payload = {
    'key': config.GMAPS_API_KEY,
    'type': 'lodging', # hotels
    'radius': 5000 # meters
}

hotels.head(2)

,ID,City,Country,Latitude,Longitude,Time,Description,Max Temp,Humidity,Wind Speed,Cloudiness,Rain,Snow,Hotel
2,2,Butaritari,KI,3.0707,172.7902,2021-01-09 20:45:49,light rain,81.70,80,21.07,62,0.2200,0.0,NaN
26,26,San Policarpo,PH,12.1791,125.5072,2021-01-09 20:45:54,heavy intensity rain,80.11,85,19.33,100,5.0126,0.0,NaN


In [8]:
# Sample response
payload['location'] = f'{hotels.iloc[0, 3]},{hotels.iloc[0, 4]}' # first city in data
response = requests.get(base_url, params=payload)
response.json()

{'html_attributions': [],
 'results': [{'business_status': 'OPERATIONAL',
   'geometry': {'location': {'lat': 3.071017, 'lng': 172.7899308},
    'viewport': {'northeast': {'lat': 3.072182880291502,
      'lng': 172.7914287802915},
     'southwest': {'lat': 3.069484919708498, 'lng': 172.7887308197085}}},
   'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png',
   'name': 'Isles Sunset Lodge',
   'photos': [{'height': 3024,
     'html_attributions': ['<a href="https://maps.google.com/maps/contrib/108849642341079970403">Ricardo Torres Coll</a>'],
     'photo_reference': 'ATtYBwI8R7xCjI1TEEZl4FX2kB65HDuog59gIXb-R9MbTahtDHpbjMTPU5xAumc_T3IOTPYWy_QiYF8FPNSxRtqHkEiD91OTXEVN-g5iCjF0XibRV4fzwcYH_Fhv40mxQiHyKp9j5gbrk4j8714y0h1u_frurapv-mWuCEyPUfRlcl5kWdhj',
     'width': 4032}],
   'place_id': 'ChIJG66HC9itb2URsiEXDJRiUHw',
   'plus_code': {'global_code': '6VMJ3QCQ+CX'},
   'rating': 4,
   'reference': 'ChIJG66HC9itb2URsiEXDJRiUHw',
   'scope': 'GOOGLE',
   'types

In [9]:
# Indices of cities without results
i404 = []

# Find nearest hotel for each city
for idx, row in hotels.iterrows():
    payload['location'] = f'{row["Latitude"]},{row["Longitude"]}' # city coordinates
    try:
        response = requests.get(base_url, params=payload)
        response_json = response.json()
        hotels.loc[idx, 'Hotel'] = response_json['results'][0]['name']
    except:
        i404.append(idx)

print(len(i404))
hotels.loc[i404, 'City'].tolist()

6


['Mabaruma', 'Mitú', 'Barra Patuca', 'Amapá', 'Ndendé', 'Micheweni']

In [10]:
# Drop cities without results
hotels.dropna(inplace=True)
print(hotels.shape)
hotels.head(2)

(64, 14)


,ID,City,Country,Latitude,Longitude,Time,Description,Max Temp,Humidity,Wind Speed,Cloudiness,Rain,Snow,Hotel
2,2,Butaritari,KI,3.0707,172.7902,2021-01-09 20:45:49,light rain,81.70,80,21.07,62,0.2200,0.0,Isles Sunset Lodge
26,26,San Policarpo,PH,12.1791,125.5072,2021-01-09 20:45:54,heavy intensity rain,80.11,85,19.33,100,5.0126,0.0,JM Lodge


### Temperature heatmap

In [11]:
# For Jupyterlab
# !jupyter labextension install @jupyter-widgets/jupyterlab-manager
# !jupyter lab build

In [12]:
# Locations and measurements
locations = hotels[['Latitude', 'Longitude']]
weights = hotels['Max Temp'].apply(lambda t: max(t, 0))

# Info
hotel_info = """
<dl>
<dt>Hotel</dt><dd>{Hotel}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather</dt><dd>{Max Temp} F - {Description}</dd>
</dl>
"""

# Layers
heat_layer = gmaps.heatmap_layer(locations, weights, dissipating=False, point_radius=7, max_intensity=140)
mark_layer = gmaps.marker_layer(locations,
                                info_box_content=[hotel_info.format(**row) for idx, row in hotels.iterrows()])

# Heatmap
fig = gmaps.figure(center=(0.0, 0.0), zoom_level=1.75)
fig.add_layer(heat_layer)
fig.add_layer(mark_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [13]:
# Save hotel data
hotels.to_csv('data/hotels.csv', index=False)
pd.read_csv('data/hotels.csv').head(2)

,ID,City,Country,Latitude,Longitude,Time,Description,Max Temp,Humidity,Wind Speed,Cloudiness,Rain,Snow,Hotel
0,2,Butaritari,KI,3.0707,172.7902,2021-01-09 20:45:49,light rain,81.70,80,21.07,62,0.2200,0.0,Isles Sunset Lodge
1,26,San Policarpo,PH,12.1791,125.5072,2021-01-09 20:45:54,heavy intensity rain,80.11,85,19.33,100,5.0126,0.0,JM Lodge
